In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x_feature = tf.feature_column.numeric_column('f1')

In [3]:
train_features = np.array([1., 2., 3., 4.])
train_labels = np.array([1.5, 3.5, 5.5, 7.5])

def train_input_fn():
    def decode_csv(line):
        parsed_line = tf.decode_csv(line, [[0.], [0.]])
        label = parsed_line[-1:]  # Last element is the label
        del parsed_line[-1]       # Delete last element
        features = parsed_line
        d = dict(zip(["f1"], features))
        return d, tf.squeeze(label)
    dataset = tf.data.TextLineDataset('train.csv').map(decode_csv)
    dataset = dataset.shuffle(1000).repeat().batch(2)
    #return dataset.make_one_shot_iterator().get_next()
    return dataset

test_features = np.array([5., 6., 7.])
test_labels = np.array([9.5, 11.5, 13.5])

def test_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices(({"f1": test_features}, test_labels))
    dataset = dataset.repeat(1).batch(len(test_features)) # batch cannot be none
    return dataset

samples = np.array([8., 9.])
def predict_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices(({"f1": samples}))
    dataset = dataset.repeat(1).batch(len(samples)) # batch cannot be none
    return dataset

In [4]:
run_config = tf.estimator.RunConfig(
    model_dir='output'
)
regressor = tf.estimator.LinearRegressor(
    feature_columns=[x_feature],
    config = run_config
)

INFO:tensorflow:Using config: {'_model_dir': 'output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000C45BEB8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
def serving_input_fn():
    inputs = {}
    inputs['f1'] = tf.placeholder(shape=[None,1], dtype=tf.int32)
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

exporter = tf.estimator.FinalExporter('linear', serving_input_fn)

In [6]:
train_spec = tf.estimator.TrainSpec(train_input_fn, max_steps=2500)
eval_spec = tf.estimator.EvalSpec(test_input_fn, steps=1, exporters=[exporter]) # Note!
metrics, _ = tf.estimator.train_and_evaluate(regressor, train_spec, eval_spec)
average_loss = metrics["average_loss"]
print(f"Average loss in testing: {average_loss:.4f}")

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into output\model.ckpt.
INFO:tensorflow:loss = 68.5, step = 1
INFO:tensorflow:global_step/sec: 781.25
INFO:tensorflow:loss = 0.6663682, step = 101 (0.129 sec)
INFO:tensorflow:global_step/sec: 943.395
INFO:tensorflow:loss = 0.28996938, step = 201 (0.107 sec)
INFO:tensorflow:global_step/sec: 854.702
INFO:tensorflow:loss = 0.6131248, step = 301 (0.117 sec)
INFO:tensorflow:global

In [ ]:
!gcloud ml-engine local predict --model-dir=./output/export/linear/1542011780/ --json-instances=./f1.json

# location of saved_model.pb

In [ ]:
!rm /tools/google-cloud-sdk/lib/googlecloudsdk/command_lib/ml_engine/*.pyc # if receive bad magic number error